In [ ]:
#!pip install numpy pandas matplotlib tensorflow

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# Load stock price data
# Replace 'AAPL.csv' with the path to your CSV file containing stock price data
data = pd.read_csv('tesla_stock.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data['Close'] = data['Close/Last'].apply(lambda x:float(x.split("$")[-1]))

In [ ]:
data = data['Close'].values.reshape(-1, 1)

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [ ]:
# Split data into training and testing sets
train_size = int(0.8 * len(data_scaled))
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]

In [ ]:
# Create sequences and labels for LSTM training
def create_sequences(data, sequence_length):
    sequences, labels = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        labels.append(data[i+sequence_length])
    return np.array(sequences), np.array(labels)

In [ ]:
sequence_length = 10  # Adjust this parameter as needed
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

In [ ]:
# Build LSTM model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(sequence_length, 1), return_sequences=True),
    LSTM(32, activation='relu'),
    Dense(1)
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
# Predict on the test data
predicted_stock_prices = model.predict(X_test)

In [ ]:
# Inverse transform to get original scale
predicted_stock_prices = scaler.inverse_transform(predicted_stock_prices)
y_test = scaler.inverse_transform(y_test)

In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='True Prices')
plt.plot(predicted_stock_prices, label='Predicted Prices')
plt.title(f"Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.legend()
plt.show()